# CADS API Python client Tests

In [1]:
import cads_api_client

## Client

The client expose the APIs for both catalogue exploration and data retrieve

In [2]:
client = cads_api_client.ApiClient("http://localhost:8080/api")
client

ApiClient(url='http://localhost:8080/api')

In [3]:
client.url

'http://localhost:8080/api'

## Catalogue Exploration

In [4]:
collections = client.collections()
collections.response.url

'http://localhost:8080/api/catalogue/v1/collections'

In [5]:
# access to request response
collections.response

<Response [200]>

In [6]:
# list of available collections
collections.collection_ids()

['derived-near-surface-meteorological-variables',
 'reanalysis-era5-land-monthly-means',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-single-levels']

In [7]:
# select a collection
collection = client.collection("reanalysis-era5-pressure-levels")
collection

Collection(response=<Response [200]>)

In [8]:
# access to request response
collection.response

<Response [200]>

**explore collection information**

In [9]:
collection.json["id"]

'reanalysis-era5-pressure-levels'

In [10]:
collection.end_datetime()

datetime.datetime(2022, 7, 20, 23, 0)

In [11]:
collection.json["description"]

'**ERA5** is the fifth generation ECMWF reanalysis for the global climate and weather for the past 4 to 7 decades.\nCurrently data is available from 1950, with Climate Data Store entries for 1950-1978 (preliminary back extension) and from 1959 onwards (final release plus timely updates, this page).\nERA5 replaces the ERA-Interim reanalysis.\n\nReanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. This principle, called data assimilation, is based on the method used by numerical weather prediction centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which an updated, improved forecast is issued. Reanalysis works in the same way, but at reduced resolution to allow for the provision of a dataset spanning back several decades. Rean

## Data Retrieve

### high level API: retrieve using client API

the **retrieve** function is blocking: 
- submits the request
- waits until the requests is completed
- downloads the data

In [12]:
client.retrieve(
    collection_id="reanalysis-era5-pressure-levels",
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["1971"], 
    month="01", 
    day="25", 
    time="06:00"
)

33176215601857374894914268157826669692.grib:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

'output.grib'

In [13]:
ls output.grib

output.grib


**list of all submitted requests**

In [14]:
requests = client.get_requests()
requests.job_ids()

['67dd2659-79bb-4f26-bfbe-f2d4d3fc35d9']

### retrieve using collection API

**request submission**

In [15]:
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["1971"], 
    month="01", 
    day="25", 
    time="06:00"
)

**request monitoring**

In [17]:
remote.status

'successful'

**download** function it blocking: 
- waits until the requests is completed
- downloads the data

In [18]:
remote.download("output.grib")

220113098894721259038618506309813230772.grib:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

'output.grib'

## Retrieve advanced usage (TBD)

### explore processes

In [19]:
processing = client.retrieve_api
processing

**list of available processes**

In [25]:
processes = processing.processes()
processes.process_ids()

['derived-near-surface-meteorological-variables',
 'reanalysis-era5-land-monthly-means',
 'reanalysis-era5-pressure-levels',
 'reanalysis-era5-land',
 'reanalysis-era5-single-levels']

**select a process**

In [21]:
process = processing.process("reanalysis-era5-pressure-levels")
process

Process(response=<Response [200]>)

In [22]:
process.json

{'title': 'ERA5 hourly data on pressure levels from 1959 to present',
 'description': '**ERA5** is the fifth generation ECMWF reanalysis for the global climate and weather for the past 4 to 7 decades.\nCurrently data is available from 1950, with Climate Data Store entries for 1950-1978 (preliminary back extension) and from 1959 onwards (final release plus timely updates, this page).\nERA5 replaces the ERA-Interim reanalysis.\n\nReanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. This principle, called data assimilation, is based on the method used by numerical weather prediction centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which an updated, improved forecast is issued. Reanalysis works in the same way, but at reduced 

### submitting a process

In [28]:
status_info = process.execute(
    inputs=dict(
        product_type="reanalysis", 
        format="grib", 
        variable="temperature", 
        pressure_level="1", 
        year=["1971"], 
        month="01", 
        day="25", 
        time="06:00"
    )
)
status_info

StatusInfo(response=<Response [201]>)

### monitoring

In [33]:
remote = statusinfo.make_remote()

In [34]:
remote.status

'successful'

**Re-create a remote from the request id**

In [35]:
request_uid = remote.request_uid
request_uid

'5d3265d3-8708-4a7b-8645-06ade157a644'

In [36]:
processing.make_remote(request_uid)

Remote(url='http://localhost:8080/api/retrieve/v1/jobs/5d3265d3-8708-4a7b-8645-06ade157a644', sleep_max=120, retry_options={})

### download result

In [37]:
result = remote.build_result()
result

Results(response=<Response [200]>)

In [38]:
result.json

{'asset': {'value': {'type': 'application/x-grib',
   'href': 'http://localhost:8080/api/storage/v1/cache/33176215601857374894914268157826669692.grib',
   'file:checksum': 262207807576530987902283886641006014631,
   'file:size': 2076600,
   'file:local_path': 's3://cache/33176215601857374894914268157826669692.grib',
   'tmp:storage_options': {},
   'tmp:open_kwargs': {'mode': 'rb'}}}}

In [40]:
result.download("output.grib")

33176215601857374894914268157826669692.grib:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

'output.grib'

## ERROS 

**MISSING COLLECTION**

In [ ]:
client.collection("missing_collection")

**WRONG URL**

In [ ]:
client = cads_api_client.ApiClient("http://localhost:8080/not_found")
client.collections()